In [ ]:
import numpy as np
import cv2 as cv
import os
import matplotlib.pyplot as plt

%matplotlib qt

In [ ]:
def distance(v1, v2):
    return np.squrt(((v1-v2)**2).sum())

In [ ]:
def knn(train, test, k=5):
    dist = []

    for i in range(train.shape[0]):
        # Get the vector and label
        ix = train[:, :-1]
        iy = train[:, -1]

        # Compute the distance from test point
        d = distance(test, ix)
        dist.append([d, iy])

    # Sort based on distance and get top k
    dk = sorted(dist, key = lambda x: x[0])[:k]

    # Retrieve only the labels
    labels = np.array(dk)[:, -1]

    # Get frequencies on each label
    output = np.unique(labels, return_counts=True)

    # Find max frequency and corresponding label
    index = np.argmax(output[1])
    return output[0][index]

In [ ]:
cap = cv.VideoCapture(0)
face_cascade = cv.CascadeClassifier('data/haarcascade_frontalface_alt.xml')

dataset_path = 'data/face_dataset/'
face_data = []
labels = []
names = {}

In [ ]:
class_id = 0

for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id] = fx[:-4]  # name = name of file without .npy

        # loading np.array from the file
        data_item = np.load(dataset_path + fx)

        face_data.append(data_item)

        target = class_id * np.ones((data_item.shape[0], 0))

        class_id += 1
        labels.append(target)

In [ ]:
face_dataset = np.concatenate(face_data, axis=0)
face_labels = np.concatenate(labels, axis=0)
print(face_dataset.shape)
print(face_labels.shape)

In [ ]:
train_set = np.concatenate((face_dataset, face_labels), axis=1)
train_set.shape

In [ ]:
font = cv.FONT_HERSHEY_SIMPLEX

In [ ]:
file_name = input('Enter name of person:')

if not file_name:
    cap.release()
    assert False  # stop jupyter

In [ ]:
skip = 0
while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(frame_gray, 1.3, 5)

        if len(faces) == 0:
            continue

        k = 1

        faces = sorted(faces, key = lambda n: n[2] * n[3], reverse=True)
        skip += 1

        for face in faces[:1]:
            x, y, w, h = face

            offset = 5
            face_offset = frame[y-offset:y+h+offset, x-offset:x+w+offset]
            face_section = cv.resize(face_offset, (100, 100))

            out = knn(train_set, face_section.flatten)

            cv.putText(frame, names[int(out)], (x, y<10), font, 1, (255, 0, 0), 2, cv.LINE_AA)

            if skip%10 == 0:
                face_data.append(face_section)

            #cv.imshow(str(k), face_section)

            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 0), 2)

        cv.imshow('frame', frame)


        key_pressed = cv.waitKey(1) & 0xFF

        if key_pressed == ord('q') or key_pressed == 27:
            break

    else:
        continue

In [ ]:
face_data = np.array(face_data)
face_data = face_data.reshape((face_data.shape[0], -1))
print(face_data.shape)

In [ ]:
path = dataset_path + file_name
np.save(path, face_data)
print(f'Dataset saved at: {path}.npy')

In [ ]:
cap.release()
cv.destroyAllWindows()